In [1]:
# Some part of the code was referenced from below.
# https://github.com/pytorch/examples/tree/master/word_language_model 
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000     # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [3]:
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
print(ids.size())
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length
print(num_batches)

torch.Size([20, 46479])
1549


In [4]:
# RNN based language model
class RNNLM(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, 
                            num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        x = self.embed(x)
        
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        
        out = self.linear(out)
        return out, (h, c)        

In [5]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
print(model)
criention = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# print parameters
module_parameters = filter(lambda p: p.requires_grad, model.parameters())
for p in module_parameters:
    print(p.size())
module_parameters = filter(lambda p: p.requires_grad, model.parameters())
param_cnt = sum([np.prod(p.size()) for p in module_parameters])
print('total trainable params: %d' % param_cnt)

RNNLM(
  (embed): Embedding(10000, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear(in_features=1024, out_features=10000, bias=True)
)
torch.Size([10000, 128])
torch.Size([4096, 128])
torch.Size([4096, 1024])
torch.Size([4096])
torch.Size([4096])
torch.Size([10000, 1024])
torch.Size([10000])
total trainable params: 16256784


In [6]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

In [7]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criention(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))


Epoch [1/5], Step[0/1549], Loss: 9.2086, Perplexity: 9982.97
Epoch [1/5], Step[100/1549], Loss: 6.0185, Perplexity: 410.97
Epoch [1/5], Step[200/1549], Loss: 5.9680, Perplexity: 390.72
Epoch [1/5], Step[300/1549], Loss: 5.7744, Perplexity: 321.95
Epoch [1/5], Step[400/1549], Loss: 5.6714, Perplexity: 290.44
Epoch [1/5], Step[500/1549], Loss: 5.1324, Perplexity: 169.43
Epoch [1/5], Step[600/1549], Loss: 5.2089, Perplexity: 182.89
Epoch [1/5], Step[700/1549], Loss: 5.3481, Perplexity: 210.21
Epoch [1/5], Step[800/1549], Loss: 5.1976, Perplexity: 180.84
Epoch [1/5], Step[900/1549], Loss: 5.0130, Perplexity: 150.36
Epoch [1/5], Step[1000/1549], Loss: 5.0749, Perplexity: 159.96
Epoch [1/5], Step[1100/1549], Loss: 5.3591, Perplexity: 212.53
Epoch [1/5], Step[1200/1549], Loss: 5.1829, Perplexity: 178.20
Epoch [1/5], Step[1300/1549], Loss: 5.0702, Perplexity: 159.20
Epoch [1/5], Step[1400/1549], Loss: 4.8467, Perplexity: 127.32
Epoch [1/5], Step[1500/1549], Loss: 5.0828, Perplexity: 161.22
Epo

In [8]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [ ]:
# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')